In [2]:
import pandas as pd
from sqlalchemy.dialects.mssql.information_schema import columns
from tpot import TPOTClassifier

df = pd.read_csv('extractedMimic.csv')  # Replace with the path to your CSV file


C:\Users\parca\AppData\Roaming\Python\Python312\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [3]:
df.head()

,icustay_id,hadm_id,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,antibiotic_time_poe,...,glucose_min1,glucose_max1,glucose_mean,rrt,subject_id,hadm_id.1,icustay_id.1,urineoutput,colloid_bolus,crystalloid_bolus
0,205941,156324,28/5/2157 14:26:21,30/5/2157 14:18:24,metavision,28/5/2157 15:30:00,-0.044201,MRSA SCREEN,0,28/5/2157 00:00:00,...,40.0,202.0,87.250000,0,88883,156324,205941,0.0,NaN,250.0
1,252848,163315,29/7/2196 02:26:17,29/7/2196 12:02:39,metavision,29/7/2196 04:57:00,-0.104664,MRSA SCREEN,1,29/7/2196 00:00:00,...,182.0,231.0,206.500000,0,46154,163315,252848,0.0,NaN,250.0
2,237901,180937,14/2/2145 17:55:07,23/2/2145 12:43:43,metavision,14/2/2145 21:20:00,-0.142280,BLOOD CULTURE,0,15/2/2145 00:00:00,...,123.0,185.0,151.285714,1,42682,180937,237901,0.0,NaN,250.0
3,207491,143962,11/6/2159 12:47:02,14/6/2159 16:31:30,metavision,11/6/2159 12:11:00,0.025023,BLOOD CULTURE,0,11/6/2159 00:00:00,...,92.0,118.0,105.000000,0,45111,143962,207491,4.0,NaN,250.0
4,293063,118489,1/1/2135 17:28:33,2/1/2135 06:56:56,metavision,1/1/2135 15:55:00,0.064965,BLOOD CULTURE,0,1/1/2135 00:00:00,...,150.0,163.0,155.000000,0,56648,118489,293063,5.0,NaN,250.0


In [4]:
from sklearn.model_selection import train_test_split

# Step 1: Prepare the dataset for machine learning
# Drop the ICUSTAY_ID and ROW_ID columns, and separate features from labels
X = df[['urineoutput', 'lactate_min','Bun_mean'.lower(),'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()]]
# 'Urineoutput'.lower(), 'Lactate_min'.lower(), 'Bun_mean'.lower(), 'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()
y = df['thirtyday_expire_flag']

# Step 2: Handle missing values (example: filling with median or mean)
# This can be adjusted based on your specific needs and data distribution
X.fillna(X.median(), inplace=True)  # Fill missing values with median

# Step 3: Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (3191, 11)
Testing set shape: (1368, 11)


C:\Users\parca\AppData\Local\Temp\ipykernel_23612\884630001.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)  # Fill missing values with median


In [5]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Step 2: Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Step 3: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation metrics
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Model Accuracy: 83.19%
Confusion Matrix:
[[1038   70]
 [ 160  100]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      1108
           1       0.59      0.38      0.47       260

    accuracy                           0.83      1368
   macro avg       0.73      0.66      0.68      1368
weighted avg       0.81      0.83      0.82      1368



C:\Users\parca\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [13:45:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [6]:
df.isnull().sum()

icustay_id              0
hadm_id                 0
intime                  0
outtime                 0
dbsource                0
                     ... 
hadm_id.1               0
icustay_id.1            0
urineoutput             0
colloid_bolus        4050
crystalloid_bolus    1194
Length: 106, dtype: int64

In [8]:
from tpot import TPOTClassifier, TPOTRegressor
t = TPOTClassifier(generations=5 ,random_state=42 , verbosity=2)

t.fit(X_train, y_train)



Optimization Progress:   0%|          | 0/600 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(KNeighborsClassifier(input_matrix, n_neighbors=49, p=1, weights=uniform), bootstrap=False, criterion=entropy, max_features=0.6000000000000001, min_samples_leaf=4, min_samples_split=16, n_estimators=100)


TPOTClassifier(generations=5, random_state=42, verbosity=2)

In [15]:
from sklearn.ensemble import RandomForestClassifier


# Step 4: Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Step 6: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.8501
Confusion Matrix:
[[1078   30]
 [ 175   85]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1108
           1       0.74      0.33      0.45       260

    accuracy                           0.85      1368
   macro avg       0.80      0.65      0.68      1368
weighted avg       0.84      0.85      0.83      1368



In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Define the XGBoost classifier with initial hyperparameters
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Step 2: Define hyperparameters for tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # Controls the step size (lower = more conservative)
    'max_depth': [3, 5, 7],             # Depth of each tree (higher = more complex model)
    'n_estimators': [100, 200, 300],     # Number of boosting rounds (trees)
    'subsample': [0.7, 0.8, 1.0],       # Ratio of samples used for training each tree
    'colsample_bytree': [0.7, 0.8, 1.0] # Ratio of features used by each tree
}

# Step 3: Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Step 4: Best model from Grid Search
best_model = grid_search.best_estimator_

# Step 5: Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Step 6: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the best hyperparameters and evaluation metrics
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

# Step 7: Cross-validation on the entire training set
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy: {cv_scores.mean() * 100:.2f}%")


In [9]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Define features and target variable
X = df[['urineoutput', 'lactate_min', 'bun_mean', 'sysbp_min', 
         'metastatic_cancer', 'inr_max', 'age', 'sodium_max', 
         'aniongap_max', 'creatinine_min', 'spo2_mean']]
y = df['thirtyday_expire_flag']

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Define hyperparameters for tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 150, 200],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Best model from Grid Search
best_model = grid_search.best_estimator_

# Make predictions on the training data
y_pred = best_model.predict(X)
y_pred_proba = best_model.predict_proba(X)[:, 1]

# Evaluate the model's performance
auc_score = roc_auc_score(y, y_pred_proba)
conf_matrix = confusion_matrix(y, y_pred)
class_report = classification_report(y, y_pred, target_names=["No Expiration", "Expiration"], output_dict=True)

# Calculate 95% Confidence Interval using Bootstrapping
n_bootstraps = 1000
bootstrapped_scores = []
rng = np.random.RandomState(42)

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
    if len(np.unique(y.iloc[indices])) < 2:
        continue
    score = roc_auc_score(y.iloc[indices], y_pred_proba[indices])
    bootstrapped_scores.append(score)

# Confidence intervals
lower_bound = np.percentile(bootstrapped_scores, 2.5)
upper_bound = np.percentile(bootstrapped_scores, 97.5)

# Print the results
def print_classification_report(report):
    print("=== Model Performance Summary ===")
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Model AUC: {auc_score:.2f}")
    print(f"95% Confidence Interval for AUC: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("\n=== Detailed Classification Report ===\n")

    for label, metrics in report.items():
        if label in ["No Expiration", "Expiration"]:
            print(f"Class: {label}")
            print(f"  Precision: {metrics['precision']:.2f}")
            print(f"  Recall: {metrics['recall']:.2f}")
            print(f"  F1-Score: {metrics['f1-score']:.2f}")
            print(f"  Support: {metrics['support']}")
            print()

    print(f"Overall Accuracy: {report['accuracy']:.2f}")
    print(f"Macro Average Precision: {report['macro avg']['precision']:.2f}")
    print(f"Macro Average Recall: {report['macro avg']['recall']:.2f}")
    print(f"Macro Average F1-Score: {report['macro avg']['f1-score']:.2f}")
    print(f"Weighted Average Precision: {report['weighted avg']['precision']:.2f}")
    print(f"Weighted Average Recall: {report['weighted avg']['recall']:.2f}")
    print(f"Weighted Average F1-Score: {report['weighted avg']['f1-score']:.2f}")

    print("\n=== Interpretation ===")
    print("The model shows good precision in predicting non-expiration cases, "
          "but recall for expiration cases indicates room for improvement.")
    print("Consider exploring class weighting or additional feature engineering to improve predictions for the minority class.")

print_classification_report(class_report)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [11]:
# Required imports
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Assuming data is already loaded and preprocessed into features (X) and labels (y)
# X = df[['urineoutput', 'lactate_min', 'bun_mean', 'sysbp_min', 
#          'metastatic_cancer', 'inr_max', 'age', 'sodium_max', 
#          'aniongap_max', 'creatinine_min', 'spo2_mean']]
# y = df['thirtyday_expire_flag']


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Use SMOTE to oversample the minority class in the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Set up the XGBoost model with class weights
model = XGBClassifier(scale_pos_weight=class_weight_dict[1], random_state=42)

# Hyperparameter tuning (using grid search)
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

# Using GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_search.fit(X_train_smote, y_train_smote)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Compute metrics
auc_score = roc_auc_score(y_test, y_pred_proba)
class_report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Best Model AUC: {auc_score}")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values